<a href="https://colab.research.google.com/github/midnightripper/testing/blob/main/Variational_Auto_Encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Prerequisite

In [3]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.9 MB/s eta 0:00:00


In [22]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

Load the train and test data splits

In [23]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [24]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [25]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [26]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [27]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [10]:
input_shape = (19,)

In [28]:
# Define the sampling layer
# def sampling(args):
#     z_mean, z_log_var = args
#     epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], 8), mean=0., stddev=1.)
#     return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

def sampling(args):
    """Reparameterization trick. Instead of sampling from Q(z|X), 
    sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

    Parameters:
    -----------
    args: list of Tensors
        Mean and log of variance of Q(z|X)

    Returns
    -------
    z: Tensor
        Sampled latent vector
    """

    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1.0, name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z

In [29]:
encoder_inputs = keras.Input(shape=input_shape)
x = layers.Dense(64, activation="relu")(encoder_inputs)
x1 = layers.Dense(32, activation="relu")(x)
z_mean = layers.Dense(8, name="z_mean")(x1)
z_log_var = layers.Dense(8, name="z_log_var")(x1)


In [30]:
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 19)]         0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 64)           1280        ['input_3[0][0]']                
                                                                                                  
 dense_13 (Dense)               (None, 32)           2080        ['dense_12[0][0]']               
                                                                                                  
 z_mean (Dense)                 (None, 8)            264         ['dense_13[0][0]']               
                                                                                            

In [15]:
latent_inputs = keras.layers.Input(shape=(8), name='z_sampling')
x = keras.layers.Dense(32, activation='relu')(latent_inputs)
outputs = keras.layers.Dense(64, activation='sigmoid')(x)
outputs = keras.layers.Dense(19, activation='sigmoid')(outputs)
# instantiate decoder model
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 z_sampling (InputLayer)     [(None, 8)]               0         
                                                                 
 dense_2 (Dense)             (None, 32)                288       
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 19)                1235      
                                                                 
Total params: 3,635
Trainable params: 3,635
Non-trainable params: 0
_________________________________________________________________


In [35]:
print(encoder(encoder_inputs)[2])

KerasTensor(type_spec=TensorSpec(shape=(None, 8), dtype=tf.float32, name=None), name='encoder/z/add:0', description="created by layer 'encoder'")


In [16]:
outputs = decoder(encoder(encoder_inputs)[2])  # Select the Z value from outputs of the encoder
vae = keras.Model(encoder_inputs, outputs, name='vae')

In [17]:
reconstruction_loss = keras.losses.mean_squared_error(encoder_inputs, outputs)
reconstruction_loss = reconstruction_loss * 19

# KL Divergence loss
kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

Model: "vae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19)]         0           []                               
                                                                                                  
 encoder (Functional)           [(None, 8),          3888        ['input_1[0][0]']                
                                 (None, 8),                                                       
                                 (None, 8)]                                                       
                                                                                                  
 decoder (Functional)           (None, 19)           3635        ['encoder[0][2]']                
                                                                                                

In [18]:
vae.fit(x_train, epochs=50, batch_size=265, validation_data=(x_test, None))

Epoch 1/50
27/27 [==============================] - 13s 62ms/step - loss: 24.0562 - val_loss: 21.8558
Epoch 2/50
27/27 [==============================] - 1s 25ms/step - loss: 20.7982 - val_loss: 19.9371
Epoch 3/50
27/27 [==============================] - 1s 29ms/step - loss: 19.6420 - val_loss: 19.3546
Epoch 4/50
27/27 [==============================] - 1s 21ms/step - loss: 19.2667 - val_loss: 19.1624
Epoch 5/50
27/27 [==============================] - 1s 19ms/step - loss: 19.1308 - val_loss: 19.0765
Epoch 6/50
27/27 [==============================] - 0s 14ms/step - loss: 19.0742 - val_loss: 19.0325
Epoch 7/50
27/27 [==============================] - 0s 13ms/step - loss: 19.0483 - val_loss: 19.0123
Epoch 8/50
27/27 [==============================] - 0s 15ms/step - loss: 19.0765 - val_loss: 18.9982
Epoch 9/50
27/27 [==============================] - 0s 16ms/step - loss: 19.0884 - val_loss: 18.9828
Epoch 10/50
27/27 [==============================] - 0s 17ms/step - loss: 18.9833 - val_lo

In [19]:
encoder = Model(encoder_inputs, z_mean)
encoder.trainable = False
encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19)]              0         
                                                                 
 dense (Dense)               (None, 64)                1280      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 z_mean (Dense)              (None, 8)                 264       
                                                                 
Total params: 3,624
Trainable params: 0
Non-trainable params: 3,624
_________________________________________________________________


In [20]:
classifier_input = Input(shape=(19,))
encoded_input = encoder(classifier_input)
x1=Dense(128,activation='relu')(encoded_input)
x2=Dense(64,activation='relu')(x1)
x3=Dense(32,activation='relu')(x2)
x4=Dense(16,activation='relu')(x3)
x5=Dense(8,activation='relu')(x4)
x6=Dense(4,activation='relu')(x5)
output_layer = Dense(1, activation='sigmoid')(x6)

classifier = Model(classifier_input, output_layer)
classifier.summary()
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 19)]              0         
                                                                 
 model (Functional)          (None, 8)                 3624      
                                                                 
 dense_5 (Dense)             (None, 128)               1152      
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 8)                 136 

In [21]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

196/196 [==============================] - 1s 6ms/step - loss: 0.5869 - accuracy: 0.6828
Test accuracy: 68.28%


In [ ]:
accuracy = classifier.evaluate(x_train, y_train)[1]
print(f"Train accuracy: {round(accuracy * 100, 2)}%")

219/219 [==============================] - 1s 3ms/step - loss: 0.3567 - accuracy: 0.8431
Train accuracy: 84.31%
